<h1>1. Импорт библиотек</h1>

In [1]:
import glob
import os
import pandas as pd
import numpy as np
import json
import docx
from bs4 import BeautifulSoup as bs
import requests
from tqdm.auto import tqdm, trange
from pymystem3 import Mystem
from nltk.stem.snowball import SnowballStemmer 
from nltk.tokenize import word_tokenize
from nltk.stem import *
from nltk.corpus import stopwords
import pyLDAvis
import pyLDAvis.lda_model
import string
import re
import pymorphy2

<h2>1.1 Парсинг данных</h2>

<h3>1.1.1 Парсинг json-файлов</h3>

Получение списка всех json-файлов

In [2]:
json_files = [f for f in os.listdir('Data\\') if f.endswith('.json')]

In [3]:
json_files

['Cybersport.Metaratings.json',
 'ESforce Holding.json',
 'Gett.json',
 'Goose Gaming.json',
 'MMA.Metaratings.json',
 'skillbox.json',
 'Vinci Agency.json',
 'Гэллэри Сервис.json',
 'Здоровье.ру.json',
 'Иннотех.json',
 'Проект по использованию технологий компьютерного зрения на базе искусственного интеллекта для анализа медицинских изображений.json',
 'Промобот.json',
 'Студия Артемия Лебедева.json',
 'Федерация креативных индустрий.json']

Словарь для заполнения набором данных с json-файлов

In [4]:
output = {
          'namecompany': [],
          'text': [],
          'ratingarticle': [],
          'date': [],
          'rating': [],
          'industries': [],
          'about': []
         }

<h3>Парсинг json-файлов</h3>

In [5]:
for file_name in json_files:
    with open(f"Data/{file_name}", "r", encoding="utf-8") as file:
        articles_json = file.read()
    articles = json.loads(articles_json)
    if articles.get('info') is not None: 
        for info in articles['info']:
            try:
                output['rating'].append(info['rate']) 
                output['industries'].append(info['industries']) 
                output['about'].append(info['about']) 
            except TypeError:
                output['rating'].append(np.nan)
                output['industries'].append(np.nan)
                output['about'].append(np.nan)
        for ref in articles['refs'][len(articles['info']):]: 
            output['rating'].append(np.nan)
            output['industries'].append(np.nan)
            output['about'].append(np.nan)
    else:
        for ref in articles['refs']:
            output['rating'].append(np.nan)
            output['industries'].append(np.nan)
            output['about'].append(np.nan)
    for ref in articles['refs']:
        if(ref is not None):
            output['namecompany'].append(file_name.split('.')[0]) 
            output['text'].append(ref[0]) 
            output['ratingarticle'].append(ref[2]['rate']) 
            output['date'].append(f"{ref[1]['day']} {ref[1]['month']} {ref[1]['time']}") 
        else:
            output['namecompany'].append(np.nan)
            output['text'].append(np.nan)
            output['ratingarticle'].append(np.nan)
            output['date'].append(np.nan)

In [6]:
len(output['about'])

1116

In [7]:
len(output['text'])

1116

In [8]:
file_name = f"articles.csv"
df = pd.DataFrame(data=output)
df.to_csv(file_name, encoding="utf-8")

In [9]:
df

,namecompany,text,ratingarticle,date,rating,industries,about
0,Cybersport,7 декабря состоялась церемония награждения лау...,6.0,8 декабря 09:30,NaN,NaN,NaN
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,6.0,8 декабря 09:30,NaN,NaN,NaN
2,ESforce Holding,Из совместного исследования издателя компьютер...,12.0,29 июля 13:50,NaN,NaN,NaN
3,ESforce Holding,По статистике The Esports Observer за I полови...,18.0,9 июля 09:33,NaN,NaN,NaN
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11.0,11 апреля 12:43,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1111,Федерация креативных индустрий,"Привет, Хабр! Представляю вашему вниманию пере...",32.0,30 января 10:43,NaN,NaN,NaN
1112,Федерация креативных индустрий,Источник\n«Умный человек не делает все ошибки ...,3.0,24 апреля 12:37,NaN,NaN,NaN
1113,Федерация креативных индустрий,"Привет, Хабр! Сегодня мы решили поделиться с с...",19.0,12 октября 14:09,NaN,NaN,NaN
1114,Федерация креативных индустрий,"Привет! Меня зовут Никита Бокарев, я креативны...",52.0,4 сентября 16:44,NaN,NaN,NaN


<h3>Вывод 1.1.1</h3>

Набор данных из json-файлов содержит много записей, в которых отсутствуют данные о компаниях, такие как рейтинг, отрасли и описание.

<h3>1.1.2 Парсинг с сайта habr</h3>

In [10]:
document = docx.Document("Condidates.docx")

Заполнение списка компаний, которые учавствуют в конкурсе из docx-файла Condidates

In [11]:
companies = [para.text.strip() for para in document.paragraphs if para.text != '']
companies

['Здоровье.ру',
 'Проект по использованию технологий компьютерного зрения на базе искусственного интеллекта (ИИ) для анализа медицинских изображений',
 'Skillbox',
 'Gett',
 'Промобот',
 'Иннотех',
 'Cybersport.Metaratings',
 'MMA.Metaratings',
 'Goose Gaming',
 'ESforce Holding',
 'Vinci Agency',
 'Гэллэри Сервис',
 'Студия Артемия Лебедева',
 'Федерация креативных индустрий',
 'Метарейтинг',
 'СберМаркет',
 'Balance Platform',
 'Московская биржа',
 'Samsung Electronics',
 'Нетология',
 'Дневник МЭШ',
 'Цифровое образование']

Список названий компаний, у которых нет информации в json-файлах

In [12]:
compnotjson = [comp for comp in companies if f"{comp}.json" not in json_files]
print(compnotjson)

['Проект по использованию технологий компьютерного зрения на базе искусственного интеллекта (ИИ) для анализа медицинских изображений', 'Skillbox', 'Метарейтинг', 'СберМаркет', 'Balance Platform', 'Московская биржа', 'Samsung Electronics', 'Нетология', 'Дневник МЭШ', 'Цифровое образование']


In [13]:
output = {
          'namecompany': [],
          'text': [],
          'ratingarticle': [],
          'date': [],
          'rating': [],
          'industries': [],
          'about': []
         }

<h3>Парсинг с сайта</h3>

In [14]:
import threading
from queue import Queue

def process_company(company):
    
    url = f"https://habr.com/ru/search/?q={company}&target_type=companies&order=relevance"
    page = requests.get(url)
    soup = bs(page.text, "html.parser")
    
    temp = soup.find("div", class_ = "tm-company-snippet__info")

    if temp is not None:
        # Переход на страницу профиля компании
        url = f"https://habr.com{temp.a.get('href')}"
        page = requests.get(url)
        soup = bs(page.text, "html.parser")
            
        # Получение рейтинга
        rating = soup.find("span", class_ = "tm-votes-lever__score-counter tm-votes-lever__score-counter tm-votes-lever__score-counter_rating")
        rating = ''.join(rating.text.split())
            
        # Получение описания 
        about = soup.find("div", class_ = "tm-company-card__description")
        about = about.text
            
        # Получение отраслей
        field = soup.find_all("a", class_ = "tm-company-profile__categories-text")
        fieldtext = [f.text.strip() for f in field]
            
        # Получение названия
        namecompany = soup.find("a", class_ = "tm-company-card__name router-link-exact-active router-link-active")
        name = namecompany.text.strip()
            
        if name in compnotjson:
            url = f"https://habr.com/ru/search/?q={company}&target_type=posts&order=relevance"
            page = requests.get(url)
            soup = bs(page.text, "html.parser")
            
            pages = soup.find_all("a", class_ = "tm-pagination__page")
            pages = int(pages[1].text)
            
            for temp_page in range(pages):
                url = f'https://habr.com/ru/search/page{temp_page}/?q={company}&target_type=posts&order=relevance'
                search_page = requests.get(url)
                search_soup = bs(search_page.text, "html.parser")
                articles = search_soup.find_all("h2", class_ = "tm-title tm-title_h2")
                
                for article in articles:
                    url = f"https://habr.com{article.a.get('href')}"
                    article_page = requests.get(url)
                    article_soup = bs(article_page.text, "html.parser")
                    text = article_soup.find("div", class_ = "tm-article-body")
                    text = text.text.strip() if text else None
                    date_published_elem = article_soup.find("span", class_ = "tm-article-datetime-published")
                    date_published = date_published_elem.time['datetime'] if date_published_elem else None
                    
                    output['text'].append(text)
                    output['date'].append(date_published)
                    output['namecompany'].append(name)
                    output['ratingarticle'].append(None)
                    output['rating'].append(rating)
                    output['about'].append(about)
                    output['industries'].append(fieldtext)
    

def main():
    threads = []
    for _ in range(10):
        thread = threading.Thread(target=worker)
        threads.append(thread)
        thread.start()


    for company in compnotjson:
        q.put(company)

    q.join()
        
def worker():
    while True:
        company = q.get()
        
        try:
            process_company(company)
        except Exception as ex:
            print(f"Произошла ошибка во время обработки {company}: {ex}")
            
        q.task_done()


# настройки очереди для многопоточности
q = Queue()

if __name__ == '__main__':
    main()

Сохранение статей с сайта habr

In [15]:
FILE_NAME = "aticles_web.csv"
df2 = pd.DataFrame(data=output)
df2.to_csv(FILE_NAME)

In [16]:
df2

,namecompany,text,ratingarticle,date,rating,industries,about
0,Skillbox,С ноября по декабрь 2021 года в компаниях Skil...,None,2022-01-25T08:11:20.000Z,119.91,"[Веб-разработка, Дизайн и юзабилити, Реклама и...",Образовательная платформа Skillbox
1,Нетология,"Нетология запускает онлайн-магистратуры, разра...",None,2021-03-17T16:12:49.000Z,72.83,"[Веб-разработка, Дизайн и юзабилити, Реклама и...",Меняем карьеру через образование
2,СберМаркет,"СберМаркет за 1,5 года прошел путь, который за...",None,2022-02-03T14:00:02.000Z,106.93,[Электронная коммерция],Кодим будущее доставки товаров
3,Skillbox,VK задумала создать свою систему управления об...,None,2022-11-28T09:30:19.000Z,119.91,"[Веб-разработка, Дизайн и юзабилити, Реклама и...",Образовательная платформа Skillbox
4,Нетология,Издание Roem сообщает об успешно завершенном р...,None,2015-02-03T09:36:51.000Z,72.83,"[Веб-разработка, Дизайн и юзабилити, Реклама и...",Меняем карьеру через образование
...,...,...,...,...,...,...,...
929,Нетология,В течение последних нескольких лет в России ос...,None,2022-04-11T12:08:45.000Z,72.83,"[Веб-разработка, Дизайн и юзабилити, Реклама и...",Меняем карьеру через образование
930,Нетология,"Привет, Хабр! Я Юрий Ковальский, сооснователь ...",None,2022-05-05T14:21:46.000Z,72.83,"[Веб-разработка, Дизайн и юзабилити, Реклама и...",Меняем карьеру через образование
931,Нетология,Этот кейс я сделал в 2018 году и в начале 2019...,None,2022-06-01T14:38:51.000Z,72.83,"[Веб-разработка, Дизайн и юзабилити, Реклама и...",Меняем карьеру через образование
932,Нетология,В этой статье: продакт овнер – это человек пож...,None,2022-06-09T10:40:25.000Z,72.83,"[Веб-разработка, Дизайн и юзабилити, Реклама и...",Меняем карьеру через образование


In [17]:
df

,namecompany,text,ratingarticle,date,rating,industries,about
0,Cybersport,7 декабря состоялась церемония награждения лау...,6.0,8 декабря 09:30,NaN,NaN,NaN
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,6.0,8 декабря 09:30,NaN,NaN,NaN
2,ESforce Holding,Из совместного исследования издателя компьютер...,12.0,29 июля 13:50,NaN,NaN,NaN
3,ESforce Holding,По статистике The Esports Observer за I полови...,18.0,9 июля 09:33,NaN,NaN,NaN
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11.0,11 апреля 12:43,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1111,Федерация креативных индустрий,"Привет, Хабр! Представляю вашему вниманию пере...",32.0,30 января 10:43,NaN,NaN,NaN
1112,Федерация креативных индустрий,Источник\n«Умный человек не делает все ошибки ...,3.0,24 апреля 12:37,NaN,NaN,NaN
1113,Федерация креативных индустрий,"Привет, Хабр! Сегодня мы решили поделиться с с...",19.0,12 октября 14:09,NaN,NaN,NaN
1114,Федерация креативных индустрий,"Привет! Меня зовут Никита Бокарев, я креативны...",52.0,4 сентября 16:44,NaN,NaN,NaN


In [18]:
print(len(df['namecompany'].unique()))
print(len(companies))

14
22


В полученном наборе данных вошло 14 компаний из 22

<h3>Вывод 1.1.2</h3>

Получен набор данных из сайта habr для дальнейшего объединения его с набором из json-файла.

<h2>1.1.3 Объединение датафреймов и описание наборов данных</h2>

Объединение датафреймов

In [19]:
merged_df = pd.concat([df, df2], ignore_index=True)

In [20]:
merged_df

,namecompany,text,ratingarticle,date,rating,industries,about
0,Cybersport,7 декабря состоялась церемония награждения лау...,6.0,8 декабря 09:30,NaN,NaN,NaN
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,6.0,8 декабря 09:30,NaN,NaN,NaN
2,ESforce Holding,Из совместного исследования издателя компьютер...,12.0,29 июля 13:50,NaN,NaN,NaN
3,ESforce Holding,По статистике The Esports Observer за I полови...,18.0,9 июля 09:33,NaN,NaN,NaN
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11.0,11 апреля 12:43,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2045,Нетология,В течение последних нескольких лет в России ос...,NaN,2022-04-11T12:08:45.000Z,72.83,"[Веб-разработка, Дизайн и юзабилити, Реклама и...",Меняем карьеру через образование
2046,Нетология,"Привет, Хабр! Я Юрий Ковальский, сооснователь ...",NaN,2022-05-05T14:21:46.000Z,72.83,"[Веб-разработка, Дизайн и юзабилити, Реклама и...",Меняем карьеру через образование
2047,Нетология,Этот кейс я сделал в 2018 году и в начале 2019...,NaN,2022-06-01T14:38:51.000Z,72.83,"[Веб-разработка, Дизайн и юзабилити, Реклама и...",Меняем карьеру через образование
2048,Нетология,В этой статье: продакт овнер – это человек пож...,NaN,2022-06-09T10:40:25.000Z,72.83,"[Веб-разработка, Дизайн и юзабилити, Реклама и...",Меняем карьеру через образование


Сохранение объединенных датафреймов

In [21]:
merged_df.to_csv('merged.csv')

In [22]:
df = pd.read_csv('merged.csv')

In [23]:
df

,Unnamed: 0,namecompany,text,ratingarticle,date,rating,industries,about
0,0,Cybersport,7 декабря состоялась церемония награждения лау...,6.0,8 декабря 09:30,NaN,NaN,NaN
1,1,ESforce Holding,7 декабря состоялась церемония награждения лау...,6.0,8 декабря 09:30,NaN,NaN,NaN
2,2,ESforce Holding,Из совместного исследования издателя компьютер...,12.0,29 июля 13:50,NaN,NaN,NaN
3,3,ESforce Holding,По статистике The Esports Observer за I полови...,18.0,9 июля 09:33,NaN,NaN,NaN
4,4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11.0,11 апреля 12:43,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2045,2045,Нетология,В течение последних нескольких лет в России ос...,NaN,2022-04-11T12:08:45.000Z,72.83,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",Меняем карьеру через образование
2046,2046,Нетология,"Привет, Хабр! Я Юрий Ковальский, сооснователь ...",NaN,2022-05-05T14:21:46.000Z,72.83,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",Меняем карьеру через образование
2047,2047,Нетология,Этот кейс я сделал в 2018 году и в начале 2019...,NaN,2022-06-01T14:38:51.000Z,72.83,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",Меняем карьеру через образование
2048,2048,Нетология,В этой статье: продакт овнер – это человек пож...,NaN,2022-06-09T10:40:25.000Z,72.83,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",Меняем карьеру через образование


<h2>Описание набора данных</h2> <br>
<ul>
    <li><b>namecompany</b> - название компании.</li> <br>
    <li><b>about</b> - описание компании.</li> <br>
    <li><b>industries</b> - сферы деятельности компаний.</li> <br>
    <li><b>date</b> - дата публикации статьи.</li> <br>
    <li><b>text</b> - текст статьи.</li> <br>
    <li><b>ratingarticle</b> - рейтинг статьи.</li> <br>
    <li><b>rating</b> - рейтинг компании.</li>
</ul>

<h3>Вывод 1.1.3</h3> <br>
Получен набор состоящий из 2050 записей, в наборе содержатся записи с пустыми значениями - текст статьи, в дальнейшем они будут удалены.

<h2>1.2 Формирование структуры набора данных</h2>

<h3>1.2.1 Удаление атрибутов</h3>

Удаление атрибутов обосновывается тем, что в наборе данных есть признаки, которые не содержат важной информации. Например, атрибуты 'Unnamed: 0', 'ratingarticle', 'about' и 'date' не содержат информацию, которая была бы полезной для анализа. Атрибут 'Unnamed: 0' является нумерацией строк в датафрейме, которая уже есть в самом датафрейме. Поэтому, чтобы сделать данные более читаемыми и легкими для анализа, эти атрибуты были удалены из набора данных.

Удаление атрибутов

In [24]:
df = df.drop(['ratingarticle', 'Unnamed: 0'], axis = 1)

In [25]:
df = df.drop('about', axis = 1)

In [26]:
df

,namecompany,text,date,rating,industries
0,Cybersport,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,NaN,NaN
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,NaN,NaN
2,ESforce Holding,Из совместного исследования издателя компьютер...,29 июля 13:50,NaN,NaN
3,ESforce Holding,По статистике The Esports Observer за I полови...,9 июля 09:33,NaN,NaN
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11 апреля 12:43,NaN,NaN
...,...,...,...,...,...
2045,Нетология,В течение последних нескольких лет в России ос...,2022-04-11T12:08:45.000Z,72.83,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл..."
2046,Нетология,"Привет, Хабр! Я Юрий Ковальский, сооснователь ...",2022-05-05T14:21:46.000Z,72.83,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл..."
2047,Нетология,Этот кейс я сделал в 2018 году и в начале 2019...,2022-06-01T14:38:51.000Z,72.83,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл..."
2048,Нетология,В этой статье: продакт овнер – это человек пож...,2022-06-09T10:40:25.000Z,72.83,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл..."


<h3>1.2.2 Удаление строк</h3>

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2050 entries, 0 to 2049
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   namecompany  2046 non-null   object 
 1   text         2044 non-null   object 
 2   date         2046 non-null   object 
 3   rating       934 non-null    float64
 4   industries   934 non-null    object 
dtypes: float64(1), object(4)
memory usage: 80.2+ KB


In [28]:
df = df[df['text'].notnull()]

In [29]:
df.isnull().sum()

namecompany       0
text              0
date              0
rating         1112
industries     1112
dtype: int64

In [30]:
df.head()

,namecompany,text,date,rating,industries
0,Cybersport,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,NaN,NaN
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,NaN,NaN
2,ESforce Holding,Из совместного исследования издателя компьютер...,29 июля 13:50,NaN,NaN
3,ESforce Holding,По статистике The Esports Observer за I полови...,9 июля 09:33,NaN,NaN
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11 апреля 12:43,NaN,NaN


<h3>1.2.3 Обработка пустых данных</h3>

In [31]:
df['rating'].fillna(value=df['rating'].mean(), inplace=True)

C:\Users\djost\AppData\Local\Temp\ipykernel_4564\475906620.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating'].fillna(value=df['rating'].mean(), inplace=True)


In [32]:
df['industries'].fillna(df['industries'].mode()[0], inplace=True)

C:\Users\djost\AppData\Local\Temp\ipykernel_4564\444958946.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['industries'].fillna(df['industries'].mode()[0], inplace=True)


In [33]:
df.head()

,namecompany,text,date,rating,industries
0,Cybersport,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл..."
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл..."
2,ESforce Holding,Из совместного исследования издателя компьютер...,29 июля 13:50,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл..."
3,ESforce Holding,По статистике The Esports Observer за I полови...,9 июля 09:33,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл..."
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11 апреля 12:43,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл..."


In [34]:
df.isnull().sum()

namecompany    0
text           0
date           0
rating         0
industries     0
dtype: int64

In [35]:
df.to_csv('merged.csv', index=False)

<h2>1.3 Предварительная обработка текстовых данных</h2>
<br>
<h3>1.3.1 Удаление знаков препинания, чисел и пробелов</h3>

In [36]:
def remove_punctuation(text):
    # Удаление всех знаков препинания
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

def remove_numbers(text):
    # Удаление всех чисел
    return re.sub(r'\d+', ' ', text)

def remove_multiple_spaces(text):
    # Удаление всех лишних пробелов
    return re.sub(r'\s+', ' ', text.strip())

Функция remove_punctuation использует модуль string, который содержит все знаки препинания. <br> Метод maketrans создает таблицу перевода, применяемую потом методом translate, который удаляет все знаки препинания. <br><br>Функция remove_numbers использует регулярные выражения, а именно модуль re и метод re.sub для замены любого символа-цифры на пробел. <br><br>В конце, функция remove_multiple_spaces использует метод strip для удаления пробелов с обеих сторон и опять же регулярное выражение для удаления повторяющихся пробелов.

In [37]:
prep_text = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in df['text']]

In [38]:
df['prep_text'] = prep_text

C:\Users\djost\AppData\Local\Temp\ipykernel_4564\806626113.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prep_text'] = prep_text


In [39]:
df.head()

,namecompany,text,date,rating,industries,prep_text
0,Cybersport,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",декабря состоялась церемония награждения лауре...
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",декабря состоялась церемония награждения лауре...
2,ESforce Holding,Из совместного исследования издателя компьютер...,29 июля 13:50,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",из совместного исследования издателя компьютер...
3,ESforce Holding,По статистике The Esports Observer за I полови...,9 июля 09:33,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",по статистике the esports observer за i полови...
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11 апреля 12:43,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",компании superdata research и paypal провели с...


Знаки препинания, цифры и пробелы были удалены.

<h3>1.3.2 Стемминг и стоп-слова</h3>

Стемминг - это процесс нахождения основы слова путем удаления окончаний и аффиксов, сохраняя при этом значение базового слова.

В библиотеке NLTK представлено встроенное множество стоп-слов, которое может быть использовано для их удаления из текста, но так как это не полный список стоп-слов, который подходит для решения всех задач, рекомендуется его дополнить.

In [40]:
stemmer = SnowballStemmer("russian")

Добавление специальных символов в список стоп-слов для удаления их из текста.

In [41]:
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...', 'т.д.', 'т', 'д', 'это','—', '“', '”', 'который', '"', '[', ']'])

In [42]:
def stem_text(text):
    tokens = word_tokenize(text.lower())
    stemmed = [stemmer.stem(token) for token in tokens if token not in russian_stopwords]
    return " ".join(stemmed)

df['text_stem'] = df['prep_text'].apply(stem_text)

C:\Users\djost\AppData\Local\Temp\ipykernel_4564\2813136038.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_stem'] = df['prep_text'].apply(stem_text)


In [43]:
df.head()

,namecompany,text,date,rating,industries,prep_text,text_stem
0,Cybersport,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",декабря состоялась церемония награждения лауре...,декабр состоя церемон награжден лауреат прем р...
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",декабря состоялась церемония награждения лауре...,декабр состоя церемон награжден лауреат прем р...
2,ESforce Holding,Из совместного исследования издателя компьютер...,29 июля 13:50,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",из совместного исследования издателя компьютер...,совместн исследован издател компьютерн игр war...
3,ESforce Holding,По статистике The Esports Observer за I полови...,9 июля 09:33,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",по статистике the esports observer за i полови...,статистик the esports observer i половин год в...
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11 апреля 12:43,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",компании superdata research и paypal провели с...,компан superdata research paypal провел совмес...


In [44]:
# функция, удаляющая стоп-слова из текста
def remove_stop_words(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [token for token in tokens if token not in russian_stopwords and token.isalpha()]
    return ' '.join(filtered_tokens)

# применяем функцию к столбцу 'text'
df['text_sw'] = df['prep_text'].apply(remove_stop_words)

C:\Users\djost\AppData\Local\Temp\ipykernel_4564\1891103662.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_sw'] = df['prep_text'].apply(remove_stop_words)


In [45]:
word_tokenize(df['text_sw'][0])

['декабря',
 'состоялась',
 'церемония',
 'награждения',
 'лауреатов',
 'премии',
 'рунета',
 'года',
 'году',
 'премию',
 'подано',
 'работ',
 'вклад',
 'развитие',
 'российского',
 'сегмента',
 'сети',
 'интернет',
 'девять',
 'основных',
 'четыре',
 'специальные',
 'номинации',
 'финалистами',
 'конкурса',
 'каждой',
 'номинации',
 'стали',
 'организаций',
 'лауреатами',
 'премии',
 'рунета',
 'стали',
 'компании',
 'персоны',
 'причем',
 'раньше',
 'организаторы',
 'российская',
 'ассоциация',
 'электронных',
 'коммуникаций',
 'раэк',
 'награждали',
 'несколько',
 'лучших',
 'представителей',
 'каждой',
 'номинации',
 'мнению',
 'экспертного',
 'сообщества',
 'году',
 'обратились',
 'помощи',
 'искусственного',
 'интеллекта',
 'добавили',
 'одного',
 'лауреата',
 'выбирался',
 'именно',
 'мнению',
 'ии',
 'номинация',
 'государство',
 'общество',
 'победители',
 'благотворительный',
 'фонд',
 'подари',
 'жизнь',
 'ано',
 'диалог',
 'правительство',
 'москвы',
 'проект',
 'цифровая'

In [46]:
df.head()

,namecompany,text,date,rating,industries,prep_text,text_stem,text_sw
0,Cybersport,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",декабря состоялась церемония награждения лауре...,декабр состоя церемон награжден лауреат прем р...,декабря состоялась церемония награждения лауре...
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",декабря состоялась церемония награждения лауре...,декабр состоя церемон награжден лауреат прем р...,декабря состоялась церемония награждения лауре...
2,ESforce Holding,Из совместного исследования издателя компьютер...,29 июля 13:50,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",из совместного исследования издателя компьютер...,совместн исследован издател компьютерн игр war...,совместного исследования издателя компьютерных...
3,ESforce Holding,По статистике The Esports Observer за I полови...,9 июля 09:33,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",по статистике the esports observer за i полови...,статистик the esports observer i половин год в...,статистике the esports observer i половину год...
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11 апреля 12:43,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",компании superdata research и paypal провели с...,компан superdata research paypal провел совмес...,компании superdata research paypal провели сов...


In [47]:
df.drop('prep_text', axis=1, inplace=True)

C:\Users\djost\AppData\Local\Temp\ipykernel_4564\2305128118.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('prep_text', axis=1, inplace=True)


В итоге тексты статей без стоп-слов, что может использоваться в дальнейшем для лемматизации.

<h3>1.3.3 Лемматизация</h3>

In [48]:
mystem = Mystem()
# Функция, выполняющая лемматизацию текста
def lemmatize_text(text):
    tokens = [token for token in mystem.lemmatize(text) if token.isalpha() and token not in russian_stopwords]
    return " ".join(tokens)

# Лемматизация столбца 'text_sw'
df['text_lemm'] = df['text_sw'].apply(lemmatize_text)

C:\Users\djost\AppData\Local\Temp\ipykernel_4564\1420078016.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_lemm'] = df['text_sw'].apply(lemmatize_text)


In [49]:
df.head()

,namecompany,text,date,rating,industries,text_stem,text_sw,text_lemm
0,Cybersport,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",декабр состоя церемон награжден лауреат прем р...,декабря состоялась церемония награждения лауре...,декабрь состояться церемония награждение лауре...
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",декабр состоя церемон награжден лауреат прем р...,декабря состоялась церемония награждения лауре...,декабрь состояться церемония награждение лауре...
2,ESforce Holding,Из совместного исследования издателя компьютер...,29 июля 13:50,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",совместн исследован издател компьютерн игр war...,совместного исследования издателя компьютерных...,совместный исследование издатель компьютерный ...
3,ESforce Holding,По статистике The Esports Observer за I полови...,9 июля 09:33,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",статистик the esports observer i половин год в...,статистике the esports observer i половину год...,статистика the esports observer i половина год...
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11 апреля 12:43,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",компан superdata research paypal провел совмес...,компании superdata research paypal провели сов...,компания superdata research paypal проводить с...


В результате лемматизации текстов статей в набор данных были добавлены слова в начальной форме.

<h3>1.3.4 Выделение частей речи</h3> <br>
Преобразование предложения в список слов или кортежей, каждый из которых имеет вид [слово, часть речи], является процессом приведения текста к формам, которые могут быть обработаны программой.

In [50]:
morph = pymorphy2.MorphAnalyzer(lang='ru')

C:\Users\djost\anaconda3\lib\site-packages\pymorphy2\units\base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
C:\Users\djost\anaconda3\lib\site-packages\pymorphy2\units\base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
C:\Users\djost\anaconda3\lib\site-packages\pymorphy2\units\base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
C:\Users\djost\anaconda3\lib\site-packages\pymorphy2\units\base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, def

In [51]:
def get_pos_pairs(text: str):
    words = word_tokenize(text)
    text_list = [f"({words[i]}, {str(morph.parse(words[i])[0].tag.POS)})" for i in range(len(words))]
    return text_list


def process_data(df: pd.DataFrame) -> pd.DataFrame:
    pos_list = []
    for text in df['text_lemm']:
        try:
            pos_pairs = get_pos_pairs(text)
            pos_list.append(pos_pairs)
        except Exception as e:
            pos_list.append([])
            print(e)
    df['text_ps'] = pos_list
    return df


process_data(df)

C:\Users\djost\AppData\Local\Temp\ipykernel_4564\3797484068.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_ps'] = pos_list


,namecompany,text,date,rating,industries,text_stem,text_sw,text_lemm,text_ps
0,Cybersport,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",декабр состоя церемон награжден лауреат прем р...,декабря состоялась церемония награждения лауре...,декабрь состояться церемония награждение лауре...,"[(декабрь, NOUN), (состояться, INFN), (церемон..."
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",декабр состоя церемон награжден лауреат прем р...,декабря состоялась церемония награждения лауре...,декабрь состояться церемония награждение лауре...,"[(декабрь, NOUN), (состояться, INFN), (церемон..."
2,ESforce Holding,Из совместного исследования издателя компьютер...,29 июля 13:50,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",совместн исследован издател компьютерн игр war...,совместного исследования издателя компьютерных...,совместный исследование издатель компьютерный ...,"[(совместный, ADJF), (исследование, NOUN), (из..."
3,ESforce Holding,По статистике The Esports Observer за I полови...,9 июля 09:33,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",статистик the esports observer i половин год в...,статистике the esports observer i половину год...,статистика the esports observer i половина год...,"[(статистика, NOUN), (the, None), (esports, No..."
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11 апреля 12:43,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",компан superdata research paypal провел совмес...,компании superdata research paypal провели сов...,компания superdata research paypal проводить с...,"[(компания, NOUN), (superdata, None), (researc..."
...,...,...,...,...,...,...,...,...,...
2045,Нетология,В течение последних нескольких лет в России ос...,2022-04-11T12:08:45.000Z,72.830000,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",течен последн нескольк лет росс остр ощуща деф...,течение последних нескольких лет россии остро ...,течение последний несколько год россия остро о...,"[(течение, NOUN), (последний, ADJF), (нескольк..."
2046,Нетология,"Привет, Хабр! Я Юрий Ковальский, сооснователь ...",2022-05-05T14:21:46.000Z,72.830000,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",привет хабр юр ковальск соосновател сервис про...,привет хабр юрий ковальский сооснователь серви...,привет хабр юрий ковальский сооснователь серви...,"[(привет, NOUN), (хабр, NOUN), (юрий, NOUN), (..."
2047,Нетология,Этот кейс я сделал в 2018 году и в начале 2019...,2022-06-01T14:38:51.000Z,72.830000,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",кейс сдела год начал вперв опубликова блог нет...,кейс сделал году начале впервые опубликовал бл...,кейс сделать год начало впервые опубликовывать...,"[(кейс, NOUN), (сделать, INFN), (год, NOUN), (..."
2048,Нетология,В этой статье: продакт овнер – это человек пож...,2022-06-09T10:40:25.000Z,72.830000,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",стат продакт овнер – человек пож быстр решен з...,статье продакт овнер человек поживший быстрое ...,статья продакт овнер человек пожить быстрый ре...,"[(статья, NOUN), (продакт, NOUN), (овнер, NOUN..."


In [52]:
df.head()

,namecompany,text,date,rating,industries,text_stem,text_sw,text_lemm,text_ps
0,Cybersport,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",декабр состоя церемон награжден лауреат прем р...,декабря состоялась церемония награждения лауре...,декабрь состояться церемония награждение лауре...,"[(декабрь, NOUN), (состояться, INFN), (церемон..."
1,ESforce Holding,7 декабря состоялась церемония награждения лау...,8 декабря 09:30,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",декабр состоя церемон награжден лауреат прем р...,декабря состоялась церемония награждения лауре...,декабрь состояться церемония награждение лауре...,"[(декабрь, NOUN), (состояться, INFN), (церемон..."
2,ESforce Holding,Из совместного исследования издателя компьютер...,29 июля 13:50,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",совместн исследован издател компьютерн игр war...,совместного исследования издателя компьютерных...,совместный исследование издатель компьютерный ...,"[(совместный, ADJF), (исследование, NOUN), (из..."
3,ESforce Holding,По статистике The Esports Observer за I полови...,9 июля 09:33,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",статистик the esports observer i половин год в...,статистике the esports observer i половину год...,статистика the esports observer i половина год...,"[(статистика, NOUN), (the, None), (esports, No..."
4,ESforce Holding,Компании SuperData Research и PayPal провели с...,11 апреля 12:43,97.655064,"['Веб-разработка', 'Дизайн и юзабилити', 'Рекл...",компан superdata research paypal провел совмес...,компании superdata research paypal провели сов...,компания superdata research paypal проводить с...,"[(компания, NOUN), (superdata, None), (researc..."


Сохранение обработанных данных

In [53]:
df.to_csv('dataset.csv', index=False)

In [54]:
df = pd.read_csv('dataset.csv')

<h2>1.4 Поиск ключевых слов / n-грамм.<br>Векторизация текстов</h2>

<h3>1.4.1 Выбор алгоритмов</h3><br>
Выбор алгоритмов для векторизации текста включает в себя несколько подходов. Один из таких подходов - использование мешка слов (<b>Bag of Words</b>), который решает проблему размерности по одной из осей путем представления каждого документа как вектора частоты терминов. Однако, этот метод не учитывает важность терминов, ведь одно и то же слово может повторяться в документе по несколько раз.

Другой подход включает использование метода <b>TF-IDF</b> (term frequency - inverse document frequency), который учитывает важность терминов, отражая не только частоту их появления в документе, но и частоту появления во всех документах. Это позволяет балансировать между широко используемыми и узкоспециализированными терминами.

Еще один подход включает использование <b>Word Embeddings</b> - векторных представлений слов. Векторы слов могут быть использованы для алгоритмов машинного обучения и извлечения информации, так как векторы можно складывать, вычитать и сравнивать.

<h3>1.4.2 Векторизация мешок слов</h3>

In [61]:
from sklearn.feature_extraction.text import CountVectorizer

In [62]:
vectorizer = CountVectorizer(analyzer='word', stop_words=russian_stopwords, ngram_range=(1, 3), min_df=2)
count_matrix = vectorizer.fit_transform(df['text_lemm'])

In [63]:
count_matrix.shape

(2044, 729727)

In [64]:
vectorizer.get_feature_names_out()[:100]

array(['aa', 'aaa', 'aaaab', 'aaaab nzac', 'aaaab nzac yc', 'aadhaar',
       'aadhaar facebook', 'aadhaar facebook персонализировать', 'aafter',
       'aafter content', 'aafter content breadcrumb', 'aaron',
       'aatelвводим', 'aatelвводим тегикогда',
       'aatelвводим тегикогда начинать', 'ab', 'ab bio', 'ab bio id',
       'ab name', 'ab name frank', 'ab значение',
       'ab значение трудоустройство', 'ab соответствовать',
       'ab соответствовать группировать', 'ab тест',
       'ab тест количественный', 'ab тестирование',
       'ab тестирование автор', 'ab тестирование просмотр',
       'ab тестирование часть', 'ab тесто', 'ab тесто тепловой', 'abap',
       'abap groovy', 'abap groovy haskellтакже', 'abbyy',
       'abbyy skillbox', 'abbyy skillbox спортмастер', 'abbyy timeline',
       'abbyy timeline платформа', 'abbyy timeline пять', 'abbyy россия',
       'abbyy россия дмитрий', 'abbyy спортмастер',
       'abbyy спортмастер магнит', 'abc', 'abc bcd', 'abc bcd cde',


<h3>1.4.3 Векторизация TF-IDF</h3>

In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [75]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000,
                                 min_df=0.01, stop_words=russian_stopwords,
                                 ngram_range=(1,3))

In [76]:
%%time
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text_lemm'])

CPU times: total: 25.3 s
Wall time: 27.3 s


In [77]:
tfidf_matrix.shape

(2044, 10000)

In [78]:
tfidf_vectorizer.get_feature_names_out()[:100]

array(['ab', 'about', 'academy', 'action', 'active', 'add', 'adobe',
       'advanced', 'after', 'agency', 'agile', 'agima', 'ai', 'aic',
       'airbnb', 'aliexpress', 'all', 'amazon', 'an', 'analysis',
       'analytics', 'and', 'android', 'android ios', 'android studio',
       'angular', 'angularjs', 'any', 'apache', 'api', 'app', 'app store',
       'apple', 'application', 'apps', 'ar', 'architecture', 'arduino',
       'are', 'array', 'art', 'as', 'async', 'at', 'auto', 'automation',
       'avito', 'awards', 'aws', 'azure', 'backend', 'background',
       'badoo', 'bash', 'basic', 'be', 'best', 'beta', 'beyond', 'bi',
       'big', 'big data', 'black', 'block', 'bluetooth', 'body',
       'boolean', 'bootstrap', 'box', 'brains', 'break', 'build',
       'business', 'but', 'by', 'call', 'can', 'capital', 'case', 'cc',
       'cd', 'center', 'ceo', 'change', 'check', 'chrome', 'ci', 'cicd',
       'class', 'clean', 'clojure', 'cloud', 'club', 'cms', 'code',
       'code review', '

<h3>1.4.4 Векторизация Word Embeddings</h3>

In [79]:
import nltk
from gensim.models import Word2Vec

In [85]:
Bigger_list=[]
for i in df['text_lemm']:
     li = list(word_tokenize(i))
     Bigger_list.append(li)
Model= Word2Vec(Bigger_list,min_count=1,vector_size=300,workers=4)
words = list(Model.wv.index_to_key) # получение списка слов в словаре модели

In [86]:
words

['работа',
 'свой',
 'год',
 'человек',
 'время',
 'работать',
 'мочь',
 'компания',
 'новый',
 'самый',
 'очень',
 'проект',
 'становиться',
 'использовать',
 'делать',
 'первый',
 'нужно',
 'просто',
 'курс',
 'получать',
 'наш',
 'сделать',
 'каждый',
 'задача',
 'начинать',
 'понимать',
 'команда',
 'данные',
 'приложение',
 'давать',
 'например',
 'проблема',
 'система',
 'вопрос',
 'хотеть',
 'также',
 'пользователь',
 'большой',
 'код',
 'создавать',
 'хороший',
 'несколько',
 'должный',
 'обучение',
 'знать',
 'решение',
 'сайт',
 'решать',
 'помогать',
 'поэтому',
 'разработка',
 'разработчик',
 'ваш',
 'возможность',
 'говорить',
 'находить',
 'случай',
 'продукт',
 'язык',
 'часть',
 'процесс',
 'любой',
 'весь',
 'рассказывать',
 'нужный',
 'опыт',
 'программа',
 'разный',
 'информация',
 'позволять',
 'сервис',
 'день',
 'результат',
 'помощь',
 'статья',
 'пример',
 'писать',
 'место',
 'дело',
 'игра',
 'иметь',
 'инструмент',
 'количество',
 'уровень',
 'сказать',
 'что

In [91]:
len(words)

115872